In [ ]:
!pip install -q datasets sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

datasets = load_dataset('indonli')

Generating train split:   0%|          | 0/10330 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2197 [00:00<?, ? examples/s]

Generating test_lay split:   0%|          | 0/2201 [00:00<?, ? examples/s]

Generating test_expert split:   0%|          | 0/2984 [00:00<?, ? examples/s]

Dataset indonli downloaded and prepared to /root/.cache/huggingface/datasets/indonli/indonli/1.1.0/d34041bd1d1a555a4bcb4ffdb9fe904778da6f7c5343209fc1485dd68121cb62. Subsequent calls will reuse this data.


  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10330
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2197
    })
    test_lay: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2201
    })
    test_expert: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2984
    })
})

In [ ]:
from sentence_transformers import SentenceTransformer, models

word_embedding_model = models.Transformer('indobenchmark/indobert-base-p2', max_seq_length=75)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
from sentence_transformers import InputExample

train_examples = [InputExample(texts=[item['premise'], item['hypothesis']], label=item['label']) for item in datasets['train']]
dev_examples = [InputExample(texts=[item['premise'], item['hypothesis']], label=item['label']) for item in datasets['validation']]

print(train_examples[:5])
print(dev_examples[:5])

[<sentence_transformers.readers.InputExample.InputExample object at 0x7f911c25b820>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f911c25bf40>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f911b0f1550>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f911b0f1790>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f911b0f1e80>]
[<sentence_transformers.readers.InputExample.InputExample object at 0x7f9027c98d00>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f901e6394f0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f901e6395e0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f901e639640>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f901e6396a0>]


In [ ]:
import math
from sentence_transformers import losses, evaluation
from torch.utils.data import DataLoader

train_batch_size = 16

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)

# Evaluate the original model on the STS benchmark dataset
dev_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(
    dev_examples,
    show_progress_bar=True,
    batch_size=train_batch_size,
    name='sts-dev'
)

num_epochs = 1
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) # 10% of train data for warm-up
evaluation_steps = int(len(train_dataloader) * 0.1)

model_save_path = 'indobert-finetuned-indonli'

#Tune the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    evaluator=dev_evaluator,
    evaluation_steps=evaluation_steps,
    output_path=model_save_path
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/646 [00:00<?, ?it/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

In [ ]:
from sentence_transformers import util

#Our sentences we like to encode
sentences1 = ['Topi saya bundar', 'Kalau tidak bundar.']
sentences2 = ['Bundar topi saya', 'bukan topi saya.']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

Topi saya bundar 		 Bundar topi saya 		 Score: 0.9245
Kalau tidak bundar. 		 bukan topi saya. 		 Score: 0.2025


In [ ]:
from sentence_transformers import SentenceTransformer

test_samples = [InputExample(texts=[item['premise'], item['hypothesis']], label=item['label']) for item in datasets['test_expert']]

trained_model = SentenceTransformer(model_save_path)
test_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(
    test_samples,
    batch_size=train_batch_size,
    show_progress_bar=True,
    name='sts-test'
)
test_evaluator(trained_model, output_path=model_save_path)

0.023862140128058987